In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # backend no interactivo (guardar a PNG)
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
from scipy.signal import decimate

# =============== 1) CONFIGURACIÓN ===============

audio_path      = "audios/Rec065.wav"                 # Cambia por tu archivo
tabla_path      = "tiempo_recortes_sincronizados.xlsx"
etiquetas_dir   = "etiquetas"                         # 'IDDSI{n}_{S/P}{num}.txt'
selected_iddsi  = 2

# Parámetros DWT
wavelet_name    = "db8"
J_levels        = 11                                   # con fs=22050 → A11 ≈ 0–5.38 Hz
mode_bordes     = "symmetric"

# Render de figuras
down_factor_plot = 10                                  # reducción visual de puntos (>10k muestras)
dpi_orig         = 300                                 # nitidez para originales
dpi_stack        = 170                                 # nitidez para figura DWT

# Carpetas de salida
carpeta_fig_segmento = "figuras_dwt_segmento"
carpeta_fig_sorbos   = "figuras_dwt_sorbos"
os.makedirs(carpeta_fig_segmento, exist_ok=True)
os.makedirs(carpeta_fig_sorbos, exist_ok=True)

# =============== 2) TABLA DE RECORTES (SEGMENTO) ===============

tabla = pd.read_excel(tabla_path)
tabla["ID_audio"] = tabla["ID_audio"].astype(str).str.strip()
audio_name   = os.path.basename(audio_path)
filas_target = tabla[(tabla["ID_audio"] == audio_name) & (tabla["IDDSI"] == selected_iddsi)]
if filas_target.empty:
    raise ValueError("No hay segmento IDDSI solicitado para ese audio")

# =============== 3) AUDIO (SIN NORMALIZAR, CON DOWN-SAMPLING) ===============

fs, data = wavfile.read(audio_path)
if data.ndim > 1:
    data = data[:, 0]  # canal izquierdo

data = data.astype(np.float64, copy=False)

# Antialias + decimado por 2
factor = 2
data = decimate(data, factor, ftype='fir', zero_phase=True)
fs = fs // factor
print(f"Frecuencia de muestreo reducida a {fs} Hz (downsampling x{factor}).")

# =============== HELPERS ===============

def _safe_name(s: str) -> str:
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in s)

def dwt_decompose(x, wavelet='db4', level=11, mode='symmetric'):
    """Devuelve coeffs = [A_J, D_J, D_{J-1}, ..., D_1]"""
    return pywt.wavedec(x, wavelet=wavelet, level=level, mode=mode)

def dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=None,
                         wavelet='db4', mode='symmetric', target_len=None):
    """Reconstruye SOLO A_J y/o ciertos D_j."""
    J = len(coeffs) - 1
    kept = []
    for i, c in enumerate(coeffs):
        if i == 0:
            kept.append(c if keep_A else np.zeros_like(c))
        else:
            level_num = J - (i - 1)  # i=1→D_J, i=2→D_{J-1}, ..., i=J→D_1
            use = (keep_D_levels is not None) and (level_num in keep_D_levels)
            kept.append(c if use else np.zeros_like(c))
    y = pywt.waverec(kept, wavelet=wavelet, mode=mode)
    if (target_len is not None) and (len(y) != target_len):
        y = y[:target_len]
    return y

def downsample_for_plot(sig, t, down_factor=10):
    """Reduce puntos para graficar (solo visual)."""
    n = len(sig)
    if n > 10000 and down_factor > 1:
        return sig[::down_factor], t[::down_factor]
    return sig, t

def compute_fft_db(sig, fs):
    """
    Espectro de magnitud (FFT directa) en dB de una sola cara.
    Conserva escala de amplitud alta (~100 dB).
    """
    x = sig - np.mean(sig)  # quitar componente DC
    N = len(x)
    if N == 0:
        return np.array([]), np.array([])

    # Ventana para suavizar fugas espectrales
    window = np.hanning(N)
    xw = x * window

    # FFT de una sola cara
    X = np.fft.rfft(xw)
    freqs = np.fft.rfftfreq(N, d=1.0/fs)

    # Magnitud (normalizada suavemente por la ventana)
    mag = np.abs(X) / (np.sum(window) / 2.0 + 1e-12)
    mag_db = 20 * np.log10(mag + 1e-12)     # convertir a dB (amplitud)

    return freqs, mag_db

def plot_original(seg, fs, t0, t1, title, save_path, down_factor=10, dpi=300):
    """Señal original con reducción visual para evitar 'apelotonamiento'."""
    n = len(seg)
    if n == 0:
        print(f"[WARN] Segmento vacío, no se grafica: {title}")
        return
    t = np.linspace(t0, t1, n)
    y_plot, t_plot = downsample_for_plot(seg, t, down_factor)
    plt.figure(figsize=(10, 3.2))
    plt.plot(t_plot, y_plot, linewidth=0.8, color='black')
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Amplitud (a.u.)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close()

def plot_time_and_fft(sig, fs, t0, t1, title, save_path,
                      down_factor=10, freq_max=None, freq_zoom=6000, dpi=300):
    """
    Figura con 3 subplots:
    1) Señal en el dominio del tiempo.
    2) Espectro de magnitud (Fourier) en dB – rango completo.
    3) Espectro de magnitud (Fourier) en dB – zoom hasta freq_zoom (ej. 6000 Hz).
    """
    n = len(sig)
    if n == 0:
        print(f("[WARN] Señal vacía, no se grafica FFT: {title}"))
        return

    # Tiempo
    t = np.linspace(t0, t1, n)
    y_plot, t_plot = downsample_for_plot(sig, t, down_factor)

    # FFT (una sola vez, la reutilizamos)
    freqs, mag_db = compute_fft_db(sig, fs)

    # Opcional: reducir puntos del espectro para evitar "serrucho visual"
    max_points = 4000
    if len(freqs) > max_points:
        step = len(freqs) // max_points
        freqs = freqs[::step]
        mag_db = mag_db[::step]

    fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=False)

    # ---- 1) Señal en el tiempo ----
    ax1 = axes[0]
    ax1.plot(t_plot, y_plot, linewidth=0.8, color="black")
    ax1.set_ylabel("Amplitud (a.u.)")
    ax1.set_title("Señal en el dominio del tiempo")
    ax1.grid(False)

    # ---- 2) Espectro completo ----
    ax2 = axes[1]
    ax2.plot(freqs, mag_db, linewidth=0.8)
    ax2.set_ylabel("Magnitud (dB)")
    ax2.set_title("Espectro de magnitud (Fourier) – completo")
    ax2.grid(False)
    if freq_max is not None:
        ax2.set_xlim(0, freq_max)

    # ---- 3) Espectro con zoom ----
    ax3 = axes[2]
    ax3.plot(freqs, mag_db, linewidth=0.8)
    ax3.set_xlabel("Frecuencia (Hz)")
    ax3.set_ylabel("Magnitud (dB)")
    ax3.set_title(f"Espectro con zoom (0–{freq_zoom} Hz)")
    ax3.grid(False)
    if freq_zoom is not None:
        ax3.set_xlim(0, freq_zoom)

    fig.suptitle(title, y=0.97)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

def plot_dwt_components_with_original(seg, parts, labels, fs, t0, t1, title, save_path,
                                      down_factor_orig=10, dpi=170):
    """
    Grafica: Fila 1 = señal original (con reducción visual)
             Filas siguientes = A_J, D_J, D_{J-1}, ..., D_1
    """
    n = len(parts)
    if len(seg) == 0:
        print(f"[WARN] Segmento vacío, no se grafica DWT: {title}")
        return

    t = np.linspace(t0, t1, len(seg))
    fig_h = 2.0 + 1.2 * (n + 1)
    fig, axes = plt.subplots(n + 1, 1, figsize=(10, fig_h), sharex=True)

    # ---- Señal original ----
    y0, t0p = downsample_for_plot(seg, t, down_factor_orig)
    axes[0].plot(t0p, y0, color="black", linewidth=0.8)
    axes[0].set_ylabel("Original", fontsize=9)
    axes[0].grid(False)

    # ---- Componentes DWT ----
    for ax, y, lab in zip(axes[1:], parts, labels):
        if len(y) != len(t):
            y = y[:len(t)]
        ax.plot(t, y, linewidth=1.0)
        ax.set_ylabel(lab, fontsize=9)
        ax.grid(False)

    axes[-1].set_xlabel("Tiempo (s)")
    fig.suptitle(title, y=0.995)
    fig.tight_layout(rect=[0, 0, 1, 0.98])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

# =============== 4) SEGMENTO COMPLETO ===============

row = filas_target.iloc[0]
t0_seg, t1_seg = float(row["cut_i_audio"]), float(row["cut_f_audio"])
i0_seg, i1_seg = int(t0_seg * fs), int(t1_seg * fs)
i0_seg = max(0, min(i0_seg, len(data)))
i1_seg = max(0, min(i1_seg, len(data)))
if i1_seg <= i0_seg:
    raise ValueError("Rango de segmento inválido")

segmento = data[i0_seg: i1_seg]

# Figura del segmento original
titulo_segmento = f"SEGMENTO Original – IDDSI {selected_iddsi} – {audio_name}"
ruta_segmento = os.path.join(
    carpeta_fig_segmento,
    _safe_name(f"SEG_Original_IDDSI{selected_iddsi}_{os.path.splitext(audio_name)[0]}.png")
)
plot_original(segmento, fs, t0_seg, t1_seg, titulo_segmento, ruta_segmento,
              down_factor=down_factor_plot, dpi=dpi_orig)

# Figura del SEGMENTO con FFT completa y zoom
titulo_segmento_fft = f"SEGMENTO – IDDSI {selected_iddsi} – Fourier – {audio_name}"
ruta_segmento_fft = os.path.join(
    carpeta_fig_segmento,
    _safe_name(f"SEG_Fourier_IDDSI{selected_iddsi}_{os.path.splitext(audio_name)[0]}.png")
)
plot_time_and_fft(segmento, fs, t0_seg, t1_seg,
                  titulo_segmento_fft, ruta_segmento_fft,
                  down_factor=down_factor_plot,
                  freq_max=None,     # puedes poner p.ej. 12000 si quieres
                  freq_zoom=6000,
                  dpi=dpi_orig)

# =============== 5) POR SORBO ===============

id_vol = str(row["ID_Voluntario"]).strip()
tipo = "sujetos" if "sujeto" in id_vol.lower() else "pacientes"
numero = ''.join(filter(str.isdigit, id_vol))
codigo = f"S{numero}" if tipo == "sujetos" else f"P{numero}"

sorbo_path = os.path.join(etiquetas_dir, f"IDDSI{selected_iddsi}_{codigo}.txt")
if not os.path.exists(sorbo_path):
    raise FileNotFoundError(f"No se encuentra el archivo de sorbos: {sorbo_path}")

sorbos_df = pd.read_csv(sorbo_path, sep=r"\s+")

for _, s in sorbos_df.iterrows():
    num, i_deg, f_deg = int(s["sorbo"]), float(s["i_deg"]), float(s["f_deg"])

    i0, i1 = int(i_deg * fs), int(f_deg * fs)
    i0 = max(0, min(i0, len(data)))
    i1 = max(0, min(i1, len(data)))
    if i1 <= i0:
        print(f"[WARN] Sorbo {num}: rango inválido ({i_deg}, {f_deg})")
        continue

    sorbo_sig = data[i0:i1]

    # ---- A) Señal ORIGINAL ----
    titulo_sorbo_orig = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – ORIGINAL – {audio_name}"
    ruta_sorbo_orig = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_ORIG_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_original(sorbo_sig, fs, i_deg, f_deg, titulo_sorbo_orig, ruta_sorbo_orig,
                  down_factor=down_factor_plot, dpi=dpi_orig)

    # ---- B) FOURIER (3 subplots) ----
    titulo_sorbo_fft = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – Fourier – {audio_name}"
    ruta_sorbo_fft = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_FFT_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_time_and_fft(sorbo_sig, fs, i_deg, f_deg,
                      titulo_sorbo_fft, ruta_sorbo_fft,
                      down_factor=down_factor_plot,
                      freq_max=None,
                      freq_zoom=6000,
                      dpi=dpi_orig)

    # ---- C) DWT ----
    coeffs = dwt_decompose(sorbo_sig, wavelet=wavelet_name, level=J_levels, mode=mode_bordes)
    J = len(coeffs) - 1

    comps, labels = [], []
    yA = dwt_reconstruct_only(coeffs, keep_A=True, keep_D_levels=[],
                              wavelet=wavelet_name, mode=mode_bordes, target_len=len(sorbo_sig))
    comps.append(yA)
    labels.append(f"A_{J}")

    for j in range(J, 0, -1):
        yD = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[j],
                                  wavelet=wavelet_name, mode=mode_bordes, target_len=len(sorbo_sig))
        comps.append(yD)
        labels.append(f"D_{j}")

    titulo_sorbo_dwt = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – DWT ({wavelet_name}, J={J}) – {audio_name}"
    ruta_sorbo_dwt = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_DWT_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_dwt_components_with_original(sorbo_sig, comps, labels, fs, i_deg, f_deg,
                                      titulo_sorbo_dwt, ruta_sorbo_dwt,
                                      down_factor_orig=down_factor_plot, dpi=dpi_stack)

print(f"Figuras del SEGMENTO en: {os.path.abspath(carpeta_fig_segmento)}")
print(f"Figuras por SORBO en:   {os.path.abspath(carpeta_fig_sorbos)}")


Frecuencia de muestreo reducida a 22050 Hz (downsampling x2).


c:\Users\fernando.salamanca\AppData\Local\Programs\Python\Python312\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 11 is too high: all coefficients will experience boundary effects.
  warnings.warn(


Figuras del SEGMENTO en: d:\Tesis\python\figuras_dwt_segmento
Figuras por SORBO en:   d:\Tesis\python\figuras_dwt_sorbos


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # backend no interactivo (guardar a PNG)
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
from scipy.signal import decimate

# =============== 1) CONFIGURACIÓN ===============

audio_path      = "audios/Rec065.wav"                 # Cambia por tu archivo
tabla_path      = "tiempo_recortes_sincronizados.xlsx"
etiquetas_dir   = "etiquetas"                         # 'IDDSI{n}_{S/P}{num}.txt'
selected_iddsi  = 2

# Parámetros DWT
wavelet_name    = "db8"
J_levels        = 11                                   # con fs=22050 → A11 ≈ 0–5.38 Hz
mode_bordes     = "symmetric"

# Render de figuras
down_factor_plot = 10                                  # reducción visual de puntos (>10k muestras)
dpi_orig         = 300                                 # nitidez para originales
dpi_stack        = 170                                 # nitidez para figura DWT

# Carpetas de salida
carpeta_fig_segmento = "figuras_dwt_segmento"
carpeta_fig_sorbos   = "figuras_dwt_sorbos"
os.makedirs(carpeta_fig_segmento, exist_ok=True)
os.makedirs(carpeta_fig_sorbos, exist_ok=True)

# =============== 2) TABLA DE RECORTES (SEGMENTO) ===============

tabla = pd.read_excel(tabla_path)
tabla["ID_audio"] = tabla["ID_audio"].astype(str).str.strip()
audio_name   = os.path.basename(audio_path)
filas_target = tabla[(tabla["ID_audio"] == audio_name) & (tabla["IDDSI"] == selected_iddsi)]
if filas_target.empty:
    raise ValueError("No hay segmento IDDSI solicitado para ese audio")

# =============== 3) AUDIO (SIN NORMALIZAR, CON DOWN-SAMPLING) ===============

fs, data = wavfile.read(audio_path)
if data.ndim > 1:
    data = data[:, 0]  # canal izquierdo

data = data.astype(np.float64, copy=False)

# Antialias + decimado por 2
factor = 2
data = decimate(data, factor, ftype='fir', zero_phase=True)
fs = fs // factor
print(f"Frecuencia de muestreo reducida a {fs} Hz (downsampling x{factor}).")

# =============== HELPERS ===============

def _safe_name(s: str) -> str:
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in s)

def dwt_decompose(x, wavelet='db4', level=11, mode='symmetric'):
    """Devuelve coeffs = [A_J, D_J, D_{J-1}, ..., D_1]"""
    return pywt.wavedec(x, wavelet=wavelet, level=level, mode=mode)

def dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=None,
                         wavelet='db4', mode='symmetric', target_len=None):
    """Reconstruye SOLO A_J y/o ciertos D_j."""
    J = len(coeffs) - 1
    kept = []
    for i, c in enumerate(coeffs):
        if i == 0:
            kept.append(c if keep_A else np.zeros_like(c))
        else:
            level_num = J - (i - 1)  # i=1→D_J, i=2→D_{J-1}, ..., i=J→D_1
            use = (keep_D_levels is not None) and (level_num in keep_D_levels)
            kept.append(c if use else np.zeros_like(c))
    y = pywt.waverec(kept, wavelet=wavelet, mode=mode)
    if (target_len is not None) and (len(y) != target_len):
        y = y[:target_len]
    return y

def downsample_for_plot(sig, t, down_factor=10):
    """Reduce puntos para graficar (solo visual)."""
    n = len(sig)
    if n > 10000 and down_factor > 1:
        return sig[::down_factor], t[::down_factor]
    return sig, t

def compute_fft_db(sig, fs):
    """
    Espectro de magnitud (FFT directa) en dB de una sola cara.
    Sin ventana ni normalización → magnitudes grandes (tipo ~100 dB).
    """
    x = sig - np.mean(sig)  # quitar componente DC
    N = len(x)
    if N == 0:
        return np.array([]), np.array([])

    # FFT de una sola cara, cruda
    X = np.fft.rfft(x)
    freqs = np.fft.rfftfreq(N, d=1.0/fs)

    mag = np.abs(X)
    mag_db = 20 * np.log10(mag + 1e-12)

    return freqs, mag_db

def plot_original(seg, fs, t0, t1, title, save_path, down_factor=10, dpi=300):
    """Señal original con reducción visual para evitar 'apelotonamiento'."""
    n = len(seg)
    if n == 0:
        print(f"[WARN] Segmento vacío, no se grafica: {title}")
        return
    t = np.linspace(t0, t1, n)
    y_plot, t_plot = downsample_for_plot(seg, t, down_factor)
    plt.figure(figsize=(10, 3.2))
    plt.plot(t_plot, y_plot, linewidth=0.8, color='black')
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Amplitud (a.u.)")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close()

def plot_time_and_fft(sig, fs, t0, t1, title, save_path,
                      down_factor=10, freq_max=None, freq_zoom=6000, dpi=300):
    """
    Figura con 3 subplots:
    1) Señal en el dominio del tiempo.
    2) Espectro de magnitud (Fourier) en dB – rango completo.
    3) Espectro de magnitud (Fourier) en dB – zoom hasta freq_zoom (ej. 6000 Hz).
    """
    n = len(sig)
    if n == 0:
        print(f"[WARN] Señal vacía, no se grafica FFT: {title}")
        return

    # Tiempo
    t = np.linspace(t0, t1, n)
    y_plot, t_plot = downsample_for_plot(sig, t, down_factor)

    # FFT
    freqs, mag_db = compute_fft_db(sig, fs)

    # Opcional: reducir puntos del espectro para evitar "serrucho" visual
    max_points = 4000
    if len(freqs) > max_points:
        step = len(freqs) // max_points
        freqs = freqs[::step]
        mag_db = mag_db[::step]

    fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=False)

    # ---- 1) Señal en el tiempo ----
    ax1 = axes[0]
    ax1.plot(t_plot, y_plot, linewidth=0.8, color="black")
    ax1.set_ylabel("Amplitud (a.u.)")
    ax1.set_title("Señal en el dominio del tiempo")
    ax1.grid(False)

    # ---- 2) Espectro completo ----
    ax2 = axes[1]
    ax2.plot(freqs, mag_db, linewidth=0.8)
    ax2.set_ylabel("Magnitud (dB)")
    ax2.set_title("Espectro de magnitud (Fourier) – completo")
    ax2.grid(False)
    if freq_max is not None:
        ax2.set_xlim(0, freq_max)

    # ---- 3) Espectro con zoom ----
    ax3 = axes[2]
    ax3.plot(freqs, mag_db, linewidth=0.8)
    ax3.set_xlabel("Frecuencia (Hz)")
    ax3.set_ylabel("Magnitud (dB)")
    ax3.set_title(f"Espectro con zoom (0–{freq_zoom} Hz)")
    ax3.grid(False)
    if freq_zoom is not None:
        ax3.set_xlim(0, freq_zoom)

    fig.suptitle(title, y=0.97)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

def plot_overlay_dwt_subtractions(original, signals, labels, fs, t0, t1,
                                  title, save_path, down_factor=10, dpi=300):
    """
    Superpone:
      - señal original
      - varias señales modificadas (ej. original - A_J, etc.)
    """
    n = len(original)
    if n == 0:
        print(f"[WARN] Segmento vacío, no se grafica overlay DWT: {title}")
        return

    t = np.linspace(t0, t1, n)
    y0, t_plot = downsample_for_plot(original, t, down_factor)

    plt.figure(figsize=(10, 4))
    plt.plot(t_plot, y0, linewidth=0.8, color="black", label="Original")

    for sig, lab in zip(signals, labels):
        y, _ = downsample_for_plot(sig, t, down_factor)
        plt.plot(t_plot, y, linewidth=0.8, label=lab)

    plt.xlabel("Tiempo (s)")
    plt.ylabel("Amplitud (a.u.)")
    plt.title(title)
    plt.legend(fontsize=8, loc="upper right")
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close()

def plot_dwt_components_with_original(seg, parts, labels, fs, t0, t1, title, save_path,
                                      down_factor_orig=10, dpi=170):
    """
    Grafica: Fila 1 = señal original (con reducción visual)
             Filas siguientes = A_J, D_J, D_{J-1}, ..., D_1
    """
    n = len(parts)
    if len(seg) == 0:
        print(f"[WARN] Segmento vacío, no se grafica DWT: {title}")
        return

    t = np.linspace(t0, t1, len(seg))
    fig_h = 2.0 + 1.2 * (n + 1)
    fig, axes = plt.subplots(n + 1, 1, figsize=(10, fig_h), sharex=True)

    # ---- Señal original ----
    y0, t0p = downsample_for_plot(seg, t, down_factor_orig)
    axes[0].plot(t0p, y0, color="black", linewidth=0.8)
    axes[0].set_ylabel("Original", fontsize=9)
    axes[0].grid(False)

    # ---- Componentes DWT ----
    for ax, y, lab in zip(axes[1:], parts, labels):
        if len(y) != len(t):
            y = y[:len(t)]
        ax.plot(t, y, linewidth=1.0)
        ax.set_ylabel(lab, fontsize=9)
        ax.grid(False)

    axes[-1].set_xlabel("Tiempo (s)")
    fig.suptitle(title, y=0.995)
    fig.tight_layout(rect=[0, 0, 1, 0.98])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

# =============== 4) SEGMENTO COMPLETO ===============

row = filas_target.iloc[0]
t0_seg, t1_seg = float(row["cut_i_audio"]), float(row["cut_f_audio"])
i0_seg, i1_seg = int(t0_seg * fs), int(t1_seg * fs)
i0_seg = max(0, min(i0_seg, len(data)))
i1_seg = max(0, min(i1_seg, len(data)))
if i1_seg <= i0_seg:
    raise ValueError("Rango de segmento inválido")

segmento = data[i0_seg: i1_seg]

# Figura del segmento original
titulo_segmento = f"SEGMENTO Original – IDDSI {selected_iddsi} – {audio_name}"
ruta_segmento = os.path.join(
    carpeta_fig_segmento,
    _safe_name(f"SEG_Original_IDDSI{selected_iddsi}_{os.path.splitext(audio_name)[0]}.png")
)
plot_original(segmento, fs, t0_seg, t1_seg, titulo_segmento, ruta_segmento,
              down_factor=down_factor_plot, dpi=dpi_orig)

# Figura del SEGMENTO con FFT completa y zoom
titulo_segmento_fft = f"SEGMENTO – IDDSI {selected_iddsi} – Fourier – {audio_name}"
ruta_segmento_fft = os.path.join(
    carpeta_fig_segmento,
    _safe_name(f"SEG_Fourier_IDDSI{selected_iddsi}_{os.path.splitext(audio_name)[0]}.png")
)
plot_time_and_fft(segmento, fs, t0_seg, t1_seg,
                  titulo_segmento_fft, ruta_segmento_fft,
                  down_factor=down_factor_plot,
                  freq_max=None,
                  freq_zoom=6000,
                  dpi=dpi_orig)

# =============== 5) POR SORBO ===============

id_vol = str(row["ID_Voluntario"]).strip()
tipo = "sujetos" if "sujeto" in id_vol.lower() else "pacientes"
numero = ''.join(filter(str.isdigit, id_vol))
codigo = f"S{numero}" if tipo == "sujetos" else f"P{numero}"

sorbo_path = os.path.join(etiquetas_dir, f"IDDSI{selected_iddsi}_{codigo}.txt")
if not os.path.exists(sorbo_path):
    raise FileNotFoundError(f"No se encuentra el archivo de sorbos: {sorbo_path}")

sorbos_df = pd.read_csv(sorbo_path, sep=r"\s+")

for _, s in sorbos_df.iterrows():
    num, i_deg, f_deg = int(s["sorbo"]), float(s["i_deg"]), float(s["f_deg"])

    i0, i1 = int(i_deg * fs), int(f_deg * fs)
    i0 = max(0, min(i0, len(data)))
    i1 = max(0, min(i1, len(data)))
    if i1 <= i0:
        print(f"[WARN] Sorbo {num}: rango inválido ({i_deg}, {f_deg})")
        continue

    sorbo_sig = data[i0:i1]

    # ---- A) Señal ORIGINAL ----
    titulo_sorbo_orig = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – ORIGINAL – {audio_name}"
    ruta_sorbo_orig = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_ORIG_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_original(sorbo_sig, fs, i_deg, f_deg, titulo_sorbo_orig, ruta_sorbo_orig,
                  down_factor=down_factor_plot, dpi=dpi_orig)

    # ---- B) FOURIER (3 subplots) ----
    titulo_sorbo_fft = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – Fourier – {audio_name}"
    ruta_sorbo_fft = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_FFT_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_time_and_fft(sorbo_sig, fs, i_deg, f_deg,
                      titulo_sorbo_fft, ruta_sorbo_fft,
                      down_factor=down_factor_plot,
                      freq_max=None,
                      freq_zoom=6000,
                      dpi=dpi_orig)

    # ---- C) DWT + restas específicas ----
    coeffs = dwt_decompose(sorbo_sig, wavelet=wavelet_name, level=J_levels, mode=mode_bordes)
    J = len(coeffs) - 1   # nivel máximo realmente utilizado

    # Reconstrucciones A_J, D_J y D_{J-1}
    yA = dwt_reconstruct_only(coeffs, keep_A=True, keep_D_levels=[],
                              wavelet=wavelet_name, mode=mode_bordes, target_len=len(sorbo_sig))
    yD_J = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J],
                                wavelet=wavelet_name, mode=mode_bordes, target_len=len(sorbo_sig))
    yD_Jm1 = None
    if J >= 2:
        yD_Jm1 = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J-1],
                                      wavelet=wavelet_name, mode=mode_bordes, target_len=len(sorbo_sig))

    # Señales restadas que quieres analizar
    overlay_signals = []
    overlay_labels  = []

    sig1 = sorbo_sig - yA
    overlay_signals.append(sig1)
    overlay_labels.append(f"Original - A_{J}")

    sig2 = sorbo_sig - (yA + yD_J)
    overlay_signals.append(sig2)
    overlay_labels.append(f"Original - (A_{J} + D_{J})")

    if yD_Jm1 is not None:
        sig3 = sorbo_sig - (yD_J + yD_Jm1)
        overlay_signals.append(sig3)
        overlay_labels.append(f"Original - (D_{J} + D_{J-1})")

    titulo_overlay = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – Restas DWT – {audio_name}"
    ruta_overlay = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_RESTAS_DWT_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_overlay_dwt_subtractions(sorbo_sig, overlay_signals, overlay_labels, fs,
                                  i_deg, f_deg, titulo_overlay, ruta_overlay,
                                  down_factor=down_factor_plot, dpi=dpi_orig)

    # ---- D) DWT completa (stacked) ----
    comps, labels = [], []
    comps.append(yA)
    labels.append(f"A_{J}")

    for j in range(J, 0, -1):
        yD = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[j],
                                  wavelet=wavelet_name, mode=mode_bordes, target_len=len(sorbo_sig))
        comps.append(yD)
        labels.append(f"D_{j}")

    titulo_sorbo_dwt = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – DWT ({wavelet_name}, J={J}) – {audio_name}"
    ruta_sorbo_dwt = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_DWT_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_dwt_components_with_original(sorbo_sig, comps, labels, fs, i_deg, f_deg,
                                      titulo_sorbo_dwt, ruta_sorbo_dwt,
                                      down_factor_orig=down_factor_plot, dpi=dpi_stack)

print(f"Figuras del SEGMENTO en: {os.path.abspath(carpeta_fig_segmento)}")
print(f"Figuras por SORBO en:   {os.path.abspath(carpeta_fig_sorbos)}")


Frecuencia de muestreo reducida a 22050 Hz (downsampling x2).
Figuras del SEGMENTO en: d:\Tesis\python\figuras_dwt_segmento
Figuras por SORBO en:   d:\Tesis\python\figuras_dwt_sorbos
Superposiciones A11 en: d:\Tesis\python\figuras_superp_A11


RESTAS DE SEÑALES Y SUPERPOSICION

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pywt
from scipy.io import wavfile
from scipy.signal import decimate

# ========================= CONFIGURACIÓN ==========================
audio_path      = "audios/Rec060.wav"
tabla_path      = "tiempo_recortes_sincronizados.xlsx"
etiquetas_dir   = "etiquetas"
selected_iddsi  = 2

wavelet_name    = "db8"
J_levels        = 11
mode_bordes     = "symmetric"

down_factor_plot = 10
dpi_orig         = 300
dpi_stack        = 170

carpeta_fig_segmento = "figuras_dwt_segmento"
carpeta_fig_sorbos   = "figuras_dwt_sorbos"
os.makedirs(carpeta_fig_segmento, exist_ok=True)
os.makedirs(carpeta_fig_sorbos, exist_ok=True)

# ========================= TABLA DE RECORTES ======================
tabla = pd.read_excel(tabla_path)
tabla["ID_audio"] = tabla["ID_audio"].astype(str).str.strip()
audio_name = os.path.basename(audio_path)
filas_target = tabla[(tabla["ID_audio"] == audio_name) & (tabla["IDDSI"] == selected_iddsi)]
if filas_target.empty:
    raise ValueError("No hay segmento IDDSI solicitado para ese audio")

# ========================= CARGA DE AUDIO =========================
fs, data = wavfile.read(audio_path)
if data.ndim > 1:
    data = data[:, 0]
data = data.astype(np.float64, copy=False)
data = decimate(data, 2, ftype='fir', zero_phase=True)
fs //= 2
print(f"Frecuencia de muestreo reducida a {fs} Hz (downsampling x2).")

# ========================= FUNCIONES AUXILIARES ====================
def _safe_name(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in s)

def dwt_decompose(x, wavelet='db4', level=11, mode='symmetric'):
    w = pywt.Wavelet(wavelet)
    max_level = pywt.dwt_max_level(len(x), w.dec_len)
    level_eff = min(level, max_level)
    if level_eff < level:
        print(f"[INFO] Nivel DWT reducido a {level_eff}.")
    return pywt.wavedec(x, wavelet=w, level=level_eff, mode=mode)

def dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=None,
                         wavelet='db4', mode='symmetric', target_len=None):
    J = len(coeffs) - 1
    kept = []
    for i, c in enumerate(coeffs):
        if i == 0:
            kept.append(c if keep_A else np.zeros_like(c))
        else:
            level_num = J - (i - 1)
            use = (keep_D_levels is not None) and (level_num in keep_D_levels)
            kept.append(c if use else np.zeros_like(c))
    y = pywt.waverec(kept, wavelet=wavelet, mode=mode)
    if target_len is not None and len(y) != target_len:
        y = y[:target_len]
    return y

# ========================= FFT en POTENCIA =========================
def compute_fft_power(sig, fs):
    x = sig - np.mean(sig)
    N = len(x)
    if N == 0:
        return np.array([]), np.array([])
    X = np.fft.rfft(x)
    freqs = np.fft.rfftfreq(N, 1.0 / fs)
    P = (np.abs(X) ** 2) / N
    return freqs, P

# ========================= FFT NORMAL + ZOOM ========================
def plot_fourier_restas_stacked(signals, fs, labels, colors, title,
                                save_path, zoom_path, dpi=300):

    n = len(signals)
    fig, axes = plt.subplots(n, 1, figsize=(10, 2.2*n + 2), sharex=True)
    if n == 1:
        axes = [axes]

    fig_zoom, axes_zoom = plt.subplots(n, 1, figsize=(10, 2.2*n + 2), sharex=True)
    if n == 1:
        axes_zoom = [axes_zoom]

    for sig, lab, col, ax, axz in zip(signals, labels, colors, axes, axes_zoom):
        freqs, P = compute_fft_power(sig, fs)
        P_plot = P + 1e-12   # piso para ver energía pequeña

        # Espectro completo (escala automática)
        ax.plot(freqs, P_plot, color=col, linewidth=0.8)
        ax.set_ylabel("Potencia (u.a.)")
        ax.set_title(lab)
        ax.set_ylim(bottom=0)

        # Zoom 3000 Hz – Nyquist (escala automática)
        axz.plot(freqs, P_plot, color=col, linewidth=0.8)
        axz.set_xlim(3000, fs/2)
        axz.set_ylabel("Potencia (u.a.)")
        axz.set_title(lab)
        axz.set_ylim(bottom=0)

    axes[-1].set_xlabel("Frecuencia (Hz)")
    fig.suptitle(title, y=0.98)
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

    axes_zoom[-1].set_xlabel("Frecuencia (Hz)")
    fig_zoom.suptitle(title + " (Zoom 3000 Hz–Nyquist)", y=0.98)
    fig_zoom.tight_layout(rect=[0, 0, 1, 0.96])
    fig_zoom.savefig(zoom_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig_zoom)

# ========================= DWT COMPONENTES ==========================
def plot_dwt_componentes(sig, coeffs, fs, title, save_path,
                         down_factor=1, dpi=170):
    J = len(coeffs) - 1
    N = len(sig)
    t = np.arange(N) / fs

    A_J = dwt_reconstruct_only(coeffs, keep_A=True,
                               wavelet=wavelet_name, mode=mode_bordes,
                               target_len=len(sig))

    detalles = []
    for j in range(J, 0, -1):
        d_j = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[j],
                                   wavelet=wavelet_name, mode=mode_bordes,
                                   target_len=len(sig))
        detalles.append((j, d_j))

    señales = [("Señal original", sig),
               (f"A{J}", A_J)] + [(f"D{j}", d_j) for j, d_j in detalles]

    n = len(señales)
    fig, axes = plt.subplots(n, 1, figsize=(10, 1.3*n + 2), sharex=True)
    if n == 1:
        axes = [axes]

    for ax, (lab, s) in zip(axes, señales):
        if down_factor > 1:
            s_plot = s[::down_factor]
            t_plot = t[::down_factor]
        else:
            s_plot = s
            t_plot = t
        ax.plot(t_plot, s_plot, linewidth=0.8)
        ax.set_ylabel("Amp.")
        ax.set_title(lab, fontsize=8)
        ax.grid(False)

    axes[-1].set_xlabel("Tiempo (s)")
    fig.suptitle(title, y=0.98)
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

# ====================== RECONSTRUCCIÓN SIN ERROR ====================
def plot_reconstruction_only(sig_original, sig_reconstructed, fs, title, save_path,
                             down_factor=1, dpi=200):
    N = len(sig_original)
    t = np.arange(N) / fs

    if down_factor > 1:
        t_plot = t[::down_factor]
        orig_plot = sig_original[::down_factor]
        reco_plot = sig_reconstructed[::down_factor]
    else:
        t_plot = t
        orig_plot = sig_original
        reco_plot = sig_reconstructed

    fig, ax = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

    ax[0].plot(t_plot, orig_plot, color="black", linewidth=0.8)
    ax[0].set_title("Señal original")
    ax[0].set_ylabel("Amplitud")

    ax[1].plot(t_plot, reco_plot, color="blue", linewidth=0.8)
    ax[1].set_title("Señal reconstruida (A + ΣD)")
    ax[1].set_xlabel("Tiempo (s)")
    ax[1].set_ylabel("Amplitud")

    fig.suptitle(title, y=0.98)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

# ====================== RESTAS TEMPORALES ===========================
def plot_temporal_restas(signals, labels, title, save_path,
                         fs, down_factor=1, dpi=180):

    n = len(signals)
    fig, axes = plt.subplots(n, 1, figsize=(10, 1.4*n + 2), sharex=True)
    if n == 1:
        axes = [axes]

    N = len(signals[0])
    t = np.arange(N) / fs
    t_plot = t[::down_factor] if down_factor > 1 else t

    for ax, sig, lab in zip(axes, signals, labels):
        s_plot = sig[::down_factor] if down_factor > 1 else sig
        ax.plot(t_plot, s_plot, linewidth=0.8)
        ax.set_ylabel("Amp.")
        ax.set_title(lab, fontsize=8)
        ax.grid(False)

    axes[-1].set_xlabel("Tiempo (s)")
    fig.suptitle(title, y=0.98)
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

# ========================= SEGMENTO COMPLETO =======================
row = filas_target.iloc[0]
t0_seg, t1_seg = float(row["cut_i_audio"]), float(row["cut_f_audio"])
i0_seg, i1_seg = int(t0_seg * fs), int(t1_seg * fs)
segmento = data[i0_seg:i1_seg]

# ========================= LOOP DE SORBOS ==========================
id_vol = str(row["ID_Voluntario"]).strip()
tipo = "sujetos" if "sujeto" in id_vol.lower() else "pacientes"
numero = ''.join(filter(str.isdigit, id_vol))
codigo = f"S{numero}" if tipo == "sujetos" else f"P{numero}"

sorbo_path = os.path.join(etiquetas_dir, f"IDDSI{selected_iddsi}_{codigo}.txt")
sorbos_df = pd.read_csv(sorbo_path, sep=r"\s+")

for _, s in sorbos_df.iterrows():
    num, i_deg, f_deg = int(s["sorbo"]), float(s["i_deg"]), float(s["f_deg"])
    i0, i1 = int(i_deg * fs), int(f_deg * fs)
    sorbo_sig = data[i0:i1]

    # ----------------- DWT -----------------
    coeffs = dwt_decompose(sorbo_sig, wavelet=wavelet_name,
                           level=J_levels, mode=mode_bordes)
    J = len(coeffs) - 1

    # Aproximación A_J para este sorbo
    A_J = dwt_reconstruct_only(coeffs, keep_A=True,
                               wavelet=wavelet_name, mode=mode_bordes,
                               target_len=len(sorbo_sig))

    # ----------------- RECONSTRUCCIÓN COMPLETA -----------------
    suma_total = np.zeros_like(sorbo_sig)
    for j in range(J, 0, -1):
        suma_total += dwt_reconstruct_only(coeffs, keep_A=False,
                                           keep_D_levels=[j],
                                           wavelet=wavelet_name,
                                           mode=mode_bordes,
                                           target_len=len(sorbo_sig))
    suma_total += A_J

    # ----------------- DWT COMPONENTES -----------------
    titulo_dwt = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – DWT – {audio_name}"
    ruta_dwt = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_DWT_COMPONENTES_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_dwt_componentes(sorbo_sig, coeffs, fs,
                         titulo_dwt, ruta_dwt,
                         down_factor=down_factor_plot, dpi=dpi_stack)

    # ----------------- RECONSTRUCCIÓN SOLO -----------------
    titulo_recon = f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – Reconstrucción completa – {audio_name}"
    ruta_recon = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_RECON_DWT_{os.path.splitext(audio_name)[0]}.png")
    )
    plot_reconstruction_only(sorbo_sig, suma_total, fs,
                             titulo_recon, ruta_recon,
                             down_factor=down_factor_plot, dpi=200)

    # ----------------- BANDAS INDIVIDUALES -----------------
    yD_J   = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J],
                                  wavelet=wavelet_name, mode=mode_bordes,
                                  target_len=len(sorbo_sig))
    yD_Jm1 = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J-1],
                                  wavelet=wavelet_name, mode=mode_bordes,
                                  target_len=len(sorbo_sig)) if J >= 2 else np.zeros_like(sorbo_sig)
    yD_Jm2 = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J-2],
                                  wavelet=wavelet_name, mode=mode_bordes,
                                  target_len=len(sorbo_sig)) if J >= 3 else np.zeros_like(sorbo_sig)
    yD_Jm3 = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J-3],
                                  wavelet=wavelet_name, mode=mode_bordes,
                                  target_len=len(sorbo_sig)) if J >= 4 else np.zeros_like(sorbo_sig)
    yD_Jm4 = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J-4],
                                  wavelet=wavelet_name, mode=mode_bordes,
                                  target_len=len(sorbo_sig)) if J >= 5 else np.zeros_like(sorbo_sig)
    yD_Jm5 = dwt_reconstruct_only(coeffs, keep_A=False, keep_D_levels=[J-5],
                                  wavelet=wavelet_name, mode=mode_bordes,
                                  target_len=len(sorbo_sig)) if J >= 6 else np.zeros_like(sorbo_sig)

    # ==================== RESTAS TEMPORALES ========================
    # Restas clásicas
    sig_DJ = sorbo_sig - yD_J
    sig_DJ_DJm1 = sorbo_sig - (yD_J + yD_Jm1)
    sig_DJ_DJm1_DJm2 = sorbo_sig - (yD_J + yD_Jm1 + yD_Jm2)
    sig_DJm1 = sorbo_sig - yD_Jm1
    sig_DJm2 = sorbo_sig - yD_Jm2
    sig_DJm3 = sorbo_sig - yD_Jm3
    sig_DJm4 = sorbo_sig - yD_Jm4
    sig_DJ_to_Jm4 = sorbo_sig - (yD_J + yD_Jm1 + yD_Jm2 + yD_Jm3 + yD_Jm4)

    # NUEVO: incluimos restas con AJ
    sig_AJ = sorbo_sig - A_J
    sig_AJ_DJ = sorbo_sig - (A_J + yD_J)
    sig_AJ_DJ_DJm1 = sorbo_sig - (A_J + yD_J + yD_Jm1)
    sig_AJ_DJm2 = sorbo_sig - (A_J + yD_Jm2)
    sig_AJ_all = sorbo_sig - (A_J + yD_J + yD_Jm1 + yD_Jm2 + yD_Jm3 + yD_Jm4)

    temporal_signals = [
        sorbo_sig,
        sig_DJ,
        sig_DJ_DJm1,
        sig_DJ_DJm1_DJm2,
        sig_DJm1,
        sig_DJm2,
        sig_DJm3,
        sig_DJm4,
        sig_DJ_to_Jm4,
        # Nuevas con AJ:
        sig_AJ,
        sig_AJ_DJ,
        sig_AJ_DJ_DJm1,
        sig_AJ_DJm2,
        sig_AJ_all
    ]

    temporal_labels = [
        "Señal original",
        f"Original − D{J}",
        f"Original − (D{J} + D{J-1})",
        f"Original − (D{J} + D{J-1} + D{J-2})",
        f"Original − D{J-1}",
        f"Original − D{J-2}",
        f"Original − D{J-3}",
        f"Original − D{J-4}",
        f"Original − (D{J} ... D{J-4})",
        # Nuevos:
        "Original − A_J",
        f"Original − (A_J + D{J})",
        f"Original − (A_J + D{J} + D{J-1})",
        f"Original − (A_J + D{J-2})",
        "Original − (A_J + D_J ... D_{J-4})"
    ]

    titulo_restas_temp = (
        f"{codigo} – IDDSI {selected_iddsi} – Sorbo {num} – Restas temporales (incluyendo A_J) – {audio_name}"
    )
    ruta_restas_temp = os.path.join(
        carpeta_fig_sorbos,
        _safe_name(f"{codigo}_IDDSI{selected_iddsi}_S{num}_RESTAS_TEMPORALES_AJ_{os.path.splitext(audio_name)[0]}.png")
    )

    plot_temporal_restas(
        temporal_signals, temporal_labels,
        titulo_restas_temp, ruta_restas_temp,
        fs, down_factor=down_factor_plot, dpi=170
    )

print(f" Figuras guardadas en:\n{os.path.abspath(carpeta_fig_sorbos)}")


Frecuencia de muestreo reducida a 22050 Hz (downsampling x2).
[INFO] Nivel DWT reducido a 10.
[INFO] Nivel DWT reducido a 10.
[INFO] Nivel DWT reducido a 10.
[INFO] Nivel DWT reducido a 10.
[INFO] Nivel DWT reducido a 10.
✅ Figuras guardadas en:
d:\Tesis\python\figuras_dwt_sorbos
